<a href="https://colab.research.google.com/github/pierre-brendan/PelotonIQ/blob/master/models/paris_nice_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First attempt at a model to predict the winner of Paris - Nice

In [1]:
# Mount drive to get cyclist data
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# All this code is from the Kmeans clustering code

# load modules
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing

# load the rider data
cycling_data = pd.read_csv('/content/drive/My Drive/cycling_data/rider_data.csv')
cycling_data = cycling_data[cycling_data['Rider'] != 'Bjorg Lambrecht ']
cycling_data = cycling_data[cycling_data['Rider'] != 'Robbert de Greef ']

# Load the GC data
gc_winners = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/gc_winners_consolidated.csv')
gc_winners.rename(columns = {'Cyclist':'Rider'}, inplace = True)

# Merge the GC winners to the rider data set
cycling_data = pd.merge(cycling_data, gc_winners, how='left', on='Rider')

# Load in the Monuments data
san_remo = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/san_remo.csv')
san_remo = san_remo['san_remo_win'].groupby(san_remo['Rider']).sum()
roubaix = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/roubaix.csv')
roubaix = roubaix['roubaix_win'].groupby(roubaix['Rider']).sum()
flanders = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/flanders.csv')
flanders = flanders['flanders_win'].groupby(flanders['Rider']).sum()
lombardia = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/lombardia.csv')
lombardia = lombardia['lombardia_win'].groupby(lombardia['Rider']).sum()
liege = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/liege.csv')
liege = liege['lbl_win'].groupby(liege['Rider']).sum()
strade = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/strade.csv')
strade['strade_win'] = 1
strade = strade['strade_win'].groupby(strade['Rider']).sum()
suisse = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/suisse.csv')
suisse['suisse_win'] = 1
suisse = suisse['suisse_win'].groupby(suisse['Rider']).sum()
romandie = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/romandie.csv')
romandie['romandie_win'] = 1
romandie = romandie['romandie_win'].groupby(romandie['Rider']).sum()
paris_nice = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/paris_nice.csv')
paris_nice['paris_nice_win'] = 1
paris_nice = paris_nice['paris_nice_win'].groupby(paris_nice['Rider']).sum()
gent = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/gent.csv')
gent['gent_win'] = 1
gent = gent['gent_win'].groupby(gent['Rider']).sum()
fleche = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/fleche.csv')
fleche['fleche_win'] = 1
fleche = fleche['fleche_win'].groupby(fleche['Rider']).sum()
e3_binckbank = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/e3_binckbank.csv')
e3_binckbank['e3_binckbank_win'] = 1
e3_binckbank = e3_binckbank['e3_binckbank_win'].groupby(e3_binckbank['Rider']).sum()
dauphine = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/dauphine.csv')
dauphine['dauphine_win'] = 1
dauphine = dauphine['dauphine_win'].groupby(dauphine['Rider']).sum()
amstel = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/amstel.csv')
amstel['amstel_win'] = 1
amstel = amstel['amstel_win'].groupby(amstel['Rider']).sum()


# Merge the Monuments winners to the rider data set
cycling_data = pd.merge(cycling_data, san_remo, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, flanders, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, liege, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, lombardia, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, roubaix, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, strade, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, suisse, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, fleche, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, gent, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, paris_nice, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, e3_binckbank, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, amstel, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, dauphine, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, romandie, how='left', on='Rider')

# Replace NaN with 0's
cycling_data.fillna(0, inplace=True)

# Drop country variable
cycling_data = cycling_data.drop(['Country', 'crawl_date'], axis = 1)

In [0]:
#cycling_data.describe()
# Okay, let's make a column for winners of Paris-Nice type races
# Currently I'm thinking Paris - Nice, Tour de Suisse and Dauphine (Tierrento is not in the data set currently)
cycling_data['Paris_Nice_etc'] = cycling_data['paris_nice_win'] + cycling_data['dauphine_win'] + cycling_data['suisse_win']

# Will need to drop those 3 columns now
cycling_data = cycling_data.drop(['paris_nice_win', 'dauphine_win', 'suisse_win'], axis = 1)

# Split the data sets
import numpy as np
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(cycling_data, test_size = 0.2, 
                                       random_state = 42)
# first let's make a fresh copy of our data and drop what we are trying
# to predict so it doesn't have any transmations applied to it
psg = train_set.drop('Paris_Nice_etc', axis=1)
psg_labels = train_set['Paris_Nice_etc'].copy()


# Algorithims prefer working with numbers generally, so let's convert the team
# proximity columns to numeric values
psg['Team'] = psg['Team'].astype(str)
housing_cat = psg[['Team']]
# Problem is if we use numeric examples for these variables algorithims
# will think they are near eachother. Instead we should make some one-hot
# encodings otherwise known as dummy variables for each category as it's own
# column
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OrdinalEncoder()
psg_cat_1hot = cat_encoder.fit_transform(housing_cat)

from sklearn.compose import ColumnTransformer
cat_attribs = ['Team']

full_pipeline = ColumnTransformer([
      ('cat', OneHotEncoder(), cat_attribs),
])

psg_prepared = full_pipeline.fit_transform(psg) # applying this to pandas dataframe called housing


In [36]:
#pd.DataFrame(psg_prepared)

,0
0,"(0, 18)\t1.0"
1,"(0, 21)\t1.0"
2,"(0, 3)\t1.0"
3,"(0, 59)\t1.0"
4,"(0, 28)\t1.0"
...,...
633,"(0, 33)\t1.0"
634,"(0, 34)\t1.0"
635,"(0, 26)\t1.0"
636,"(0, 92)\t1.0"


In [0]:
# Also from KMeans code
# This would be the best approach to removing special characters
#dictionary = {'ž':'z', 'č':'c', 'ü':'u', 'á':'a', 'é':'e', 'ö':'o', 'ó':'o', 'ç':'c',
#              'ń':'ń', 'í':'i', 'è':'e'}
#cycling_data.replace(dictionary, regex=True, inplace=True)

# make a df without cyclist name in it for clustering
d = psg.drop(['Rider', 'Team'], axis=1)

# let's normalize the data
d['GC'] = preprocessing.scale(d['GC'])
d['Sprint'] = preprocessing.scale(d['Sprint'])
d['TT'] = preprocessing.scale(d['TT'])
d['Climber'] = preprocessing.scale(d['Climber'])
d['Classic'] = preprocessing.scale(d['Classic'])
d['Age'] = preprocessing.scale(d['Age'])
d['Giro'] = preprocessing.scale(d['Giro'])
d['Vuelta'] = preprocessing.scale(d['Vuelta'])
d['Tour'] = preprocessing.scale(d['Tour'])
d['Total'] = preprocessing.scale(d['Total'])
d['san_remo_win'] = preprocessing.scale(d['san_remo_win'])
d['lombardia_win'] = preprocessing.scale(d['lombardia_win'])
d['flanders_win'] = preprocessing.scale(d['flanders_win'])
d['lbl_win'] = preprocessing.scale(d['lbl_win'])
d['roubaix_win'] = preprocessing.scale(d['roubaix_win'])
d['strade_win'] = preprocessing.scale(d['strade_win'])
#d['suisse_win'] = preprocessing.scale(d['suisse_win'])
d['fleche_win'] = preprocessing.scale(d['fleche_win'])
d['gent_win'] = preprocessing.scale(d['gent_win'])
#d['Paris_Nice_etc'] = preprocessing.scale(d['Paris_Nice_etc'])
d['e3_binckbank_win'] = preprocessing.scale(d['e3_binckbank_win'])
d['amstel_win'] = preprocessing.scale(d['amstel_win'])
#d['dauphine_win'] = preprocessing.scale(d['dauphine_win'])
d['romandie_win'] = preprocessing.scale(d['romandie_win'])